## Type Stability

Julia documentation says that whenever possible it helps to ensure that the return type of a function is consistent. This helps the compiler produce optimized code. Example where non type-stable code is much slower than the corresponding type stable code:

In [20]:
function repeat_unstable(x, times::Int) 
    ret = 0
    for i = 1:times
        ret += x*x
    end
end;

function repeat_stable(x, times::Int) 
    ret = 0.0
    for i = 1:times
        ret += x*x
    end
end;

In [21]:
@time repeat_unstable(0.1, 100_000)
@time repeat_stable(0.1, 100_000)

  0.000135 seconds (4 allocations: 160 bytes)
  0.000002 seconds (4 allocations: 160 bytes)


The only difference between the stable and unstable functions is the initialization of the return variable `ret`. In the unstable version it is initialized as `Int` and depending upon `x` the return type may be `Int` or `Float`.
This introduces the type instability. 

The unstable code is significantly slower we can see why this is the case by looking at the native code generated by the compiler in both the cases.  

In [10]:
@code_llvm repeat_stable(0.1, 100_000)


;  @ In[7]:9 within `repeat_stable'
define void @julia_repeat_stable_17167(double, i64) {
top:
;  @ In[7]:11 within `repeat_stable'
  ret void
}


Code generated for `repeat_stable` is straight forward as compiler can genereate a stiatically typed code (which is as fast as c). whereas the code for `repeat_unstable` needs to do a type check for the variable `ret` in every loop, and consequently the code generated is not statically typed. 


In [13]:
@code_llvm repeat_unstable(0.1, 100_000)


;  @ In[7]:2 within `repeat_unstable'
define void @julia_repeat_unstable_17166(double, i64) {
top:
;  @ In[7]:3 within `repeat_unstable'
; ┌ @ range.jl:5 within `Colon'
; │┌ @ range.jl:277 within `UnitRange'
; ││┌ @ range.jl:282 within `unitrange_last'
; │││┌ @ operators.jl:341 within `>='
; ││││┌ @ int.jl:424 within `<='
       %2 = icmp sgt i64 %1, 0
; └└└└└
  br i1 %2, label %L7.L11_crit_edge, label %L38

L7.L11_crit_edge:                                 ; preds = %top
  br label %L11

L11:                                              ; preds = %L7.L11_crit_edge, %L37
  %tindex_phi = phi i2 [ -2, %L7.L11_crit_edge ], [ 1, %L37 ]
  %value_phi2 = phi i64 [ 1, %L7.L11_crit_edge ], [ %4, %L37 ]
;  @ In[7]:4 within `repeat_unstable'
  %tindex_phi.off = add i2 %tindex_phi, -1
  %switch = icmp ult i2 %tindex_phi.off, -2
  br i1 %switch, label %L27, label %L25

L25:                                              ; preds = %L11
  call void @jl_throw(%jl_value_t addrspace(12)* addrspacecast (%

[this](https://stackoverflow.com/questions/53924454/how-does-type-stability-make-julia-so-fast) answer on stackoverflow makes it much clear:

"""
Type stability allows the compiler to determine the output types of a function directly from the input types at compile time. Because Julia specializes the compilation on every input type, this means that if all functions are type stable, the compiler can deduce the types of every value inside of a function call. When this occurs, Julia's JIT compiler will essentially create a statically typed version of the method and build the LLVM IR to compile that static version, where that static version is essentially the same assembly code as C compiled with clang (LLVM).

What this means is that, if the compiler can deduce the basic types on every variable, it will and it will emit code that forces those types to hold, just as though it was fully annotated C code. The way that Julia is dynamic is that when this doesn't hold, then it will "box" the variables, essentially creating a new type that says "I don't know what the type is" and add in code for forcing type checks and dispatch computations at runtime to handle the dynamicness. So if all type information is known, the dynamicness of Julia compiles away and ceases to even exist in the code that is ran.

This is also why multiple dispatch is so crucial to the language, since multiple dispatch then means that f(x) has a different method for f(x::Float64) (even if you don't specify the concrete version, the compiler will deduce and use that, called automatic type-specialization), and now that concrete version is more likely to be stable than the general version. Exposing multiple dispatch as a feature is really just allowing you to intercept this stage of the compilation and allow changing the type->code relationship.
"""